# 📧 Filtr Antyspamowy: Naiwny Klasyfikator Bayesa

Naiwny Bayes to algorytm, który uwielbia pracować z tekstem. Jest prosty, szybki i zaskakująco skuteczny.

**Dlaczego "Naiwny"?**
Bo patrzy na każde słowo w SMS-ie oddzielnie. Nie rozumie kontekstu (nie wie, że "Biały" i "Dom" to co innego niż "Dom" i "Biały"). Traktuje słowa jak składniki wrzucone do worka.

**Cel:**
Stworzymy model, który nauczy się rozróżniać **SPAM** (śmieci) od **HAM** (normalnych wiadomości).

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer # Do zamiany słów na liczby
from sklearn.naive_bayes import MultinomialNB # Nasz Naiwny Model

# 1. TWORZYMY DANE TRENINGOWE (Mała baza SMS-ów)
# 1 = SPAM, 0 = HAM (Normalne)
dane = {
    'tekst': [
        "Wygrałeś darmowe pieniądze kliknij",  # Spam
        "Kup teraz super oferta promocja",     # Spam
        "Odbierz nagrodę darmowe leki",        # Spam
        "Cześć idziemy jutro na kawę",         # Normalne
        "Spotkanie w biurze o 10 rano",        # Normalne
        "Mama dzwoniła pytać o obiad"          # Normalne
    ],
    'kategoria': [1, 1, 1, 0, 0, 0]
}

df = pd.DataFrame(dane)

print("--- NASZA BAZA DANYCH ---")
display(df)

--- NASZA BAZA DANYCH ---


,tekst,kategoria
0,Wygrałeś darmowe pieniądze kliknij,1
1,Kup teraz super oferta promocja,1
2,Odbierz nagrodę darmowe leki,1
3,Cześć idziemy jutro na kawę,0
4,Spotkanie w biurze o 10 rano,0
5,Mama dzwoniła pytać o obiad,0


## Krok 2: Zamiana słów na liczby (Bag of Words)

Model matematyczny nie umie czytać. Nie rozumie słowa "Wygrałeś".
Musimy zamienić tekst na macierz liczb. Użyjemy techniki **Bag of Words (Worek Słów)**.

Komputer zrobi tabelę:
*   Kolumny to wszystkie znane mu słowa (Wygrałeś, Kawa, Mama...).
*   Wartości to liczba wystąpień słowa w zdaniu.

In [2]:
# 1. Przygotowujemy "Tłumacza" (Vectorizer)
vectorizer = CountVectorizer()

# 2. Uczymy tłumacza naszych słów i zamieniamy tekst na liczby
X_matrix = vectorizer.fit_transform(df['tekst'])

# Zobaczmy, jak komputer "widzi" nasze zdania
# (Zamieniamy to z powrotem na DataFrame tylko po to, żebyś to ładnie widział)
tabela_liczbow = pd.DataFrame(X_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("--- TAK KOMPUTER WIDZI ZDANIA (Bag of Words) ---")
display(tabela_liczbow)
print("\nKażda kolumna to jedno słowo. 1 oznacza 'słowo występuje', 0 'brak słowa'.")

--- TAK KOMPUTER WIDZI ZDANIA (Bag of Words) ---


,10,biurze,cześć,darmowe,dzwoniła,idziemy,jutro,kawę,kliknij,kup,...,odbierz,oferta,pieniądze,promocja,pytać,rano,spotkanie,super,teraz,wygrałeś
0,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,1,1,0
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0



Każda kolumna to jedno słowo. 1 oznacza 'słowo występuje', 0 'brak słowa'.


## Krok 3: Trening i Testowanie Naiwnego Bayesa

Teraz wrzucamy te zera i jedynki do algorytmu `MultinomialNB`.
Model policzy statystykę:
*   "Skoro słowo 'Darmowe' pojawiło się 2 razy w Spamie i 0 razy w Normalnych, to jak zobaczę je znowu, to na 99% będzie Spam".

Przetestujemy go na nowych zdaniach, których nigdy wcześniej nie widział!

In [3]:
# 1. Trenujemy model
model = MultinomialNB()
model.fit(X_matrix, df['kategoria'])

# 2. NOWE WIADOMOŚCI DO SPRAWDZENIA
nowe_smsy = [
    "Wygrałeś darmowe spotkanie",  # Podchwytliwe: "Wygrałeś" (spam) + "Spotkanie" (normalne)
    "Cześć mamo idziemy na obiad", # Normalne
    "Kliknij po nagrodę"           # Spam
]

# Zamieniamy nowe SMS-y na liczby (używając TEGO SAMEGO tłumacza co wcześniej)
nowe_X = vectorizer.transform(nowe_smsy)

# Robimy predykcję
wynik = model.predict(nowe_X)
prawdopodobienstwo = model.predict_proba(nowe_X)

print("--- WYNIKI TESTU ---")
for i, sms in enumerate(nowe_smsy):
    werdykt = "🚨 SPAM" if wynik[i] == 1 else "✅ NORMALNA"
    pewnosc_spamu = prawdopodobienstwo[i][1] * 100 # Szansa że to spam
    
    print(f"SMS: '{sms}'")
    print(f"Werdykt: {werdykt}")
    print(f"Pewność modelu, że to SPAM: {pewnosc_spamu:.2f}%")
    print("-" * 30)

--- WYNIKI TESTU ---
SMS: 'Wygrałeś darmowe spotkanie'
Werdykt: 🚨 SPAM
Pewność modelu, że to SPAM: 75.00%
------------------------------
SMS: 'Cześć mamo idziemy na obiad'
Werdykt: ✅ NORMALNA
Pewność modelu, że to SPAM: 5.88%
------------------------------
SMS: 'Kliknij po nagrodę'
Werdykt: 🚨 SPAM
Pewność modelu, że to SPAM: 80.00%
------------------------------


## 🧠 Podsumowanie: Geniusz czy Idiota?

Dlaczego ten algorytm nazywamy "Naiwnym" i dlaczego mimo to rządzi w filtrach antyspamowych?

**Tu jest haczyk.**
Nazywamy go naiwnym, bo zachowuje się jak dziecko, które myśli, że **świat składa się z niepowiązanych klocków**.

*   Dla Bayesa zdanie *"Wygrałeś darmowe pieniądze"* to po prostu worek trzech słów: [Wygrałeś, Darmowe, Pieniądze].
*   Nie rozumie, że *"Darmowe"* i *"Pieniądze"* występujące obok siebie to znacznie silniejszy sygnał niż każde z osobna. Traktuje je jak niezależne rzuty monetą.

**Dlaczego więc to działa?**
Wyobraź sobie głosowanie.
Jeśli w zdaniu masz 10 słów typowych dla spamu (głosują na "SPAM") i 2 słowa typowe dla normalnej rozmowy (głosują na "HAM"), to nie musisz rozumieć gramatyki ani kontekstu. **Siła głosów przeważa.**

**Wniosek:**
Naiwny Bayes jest jak **szybki czytacz**, który nie wnika w sens książki, ale skanuje słowa kluczowe. Może nie zrozumie ironii, ale błyskawicznie wrzuci śmieci do kosza. Dlatego jest standardem w pracy z tekstem.